From [Julia Academy: Intro to julia](https://juliaacademy.com/courses/intro-to-julia)

# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.6541315197753956
 1.5703209755141452
 1.2189779591269714

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
```
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [7]:
A

3×3 Array{Float64,2}:
 0.420885  0.686786  0.54646
 0.361277  0.899858  0.309186
 0.170401  0.701429  0.347148

In [8]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.404863  1.0       0.0
 0.858375  0.733009  1.0
U factor:
3×3 Array{Float64,2}:
 0.420885  0.686786   0.54646
 0.0       0.423375   0.125907
 0.0       0.0       -0.252173

In [9]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [10]:
Alu.P

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [11]:
Alu.L

3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.404863  1.0       0.0
 0.858375  0.733009  1.0

In [12]:
Alu.U

3×3 Array{Float64,2}:
 0.420885  0.686786   0.54646
 0.0       0.423375   0.125907
 0.0       0.0       -0.252173

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [13]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [14]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [15]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
```

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [16]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.725339   0.55104    0.4126
 -0.622612  -0.269448  -0.734678
 -0.293662  -0.78978    0.538526
R factor:
3×3 Array{Float64,2}:
 -0.580261  -1.2644    -0.690816
  0.0       -0.417993  -0.0563588
  0.0        0.0        0.185266

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [17]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.725339   0.55104    0.4126
 -0.622612  -0.269448  -0.734678
 -0.293662  -0.78978    0.538526

In [18]:
Aqr.R

3×3 Array{Float64,2}:
 -0.580261  -1.2644    -0.690816
  0.0       -0.417993  -0.0563588
  0.0        0.0        0.185266

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [19]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 0.03963815566553452
 0.18809218402791444
 3.1080524224354096
vectors:
3×3 Array{Float64,2}:
 -0.536078   0.688581  -0.48834
 -0.157035  -0.649733  -0.743766
  0.829434   0.32203   -0.456439

The values and the vectors can be extracted from the Eigen type by special indexing

In [20]:
AsymEig.values

3-element Array{Float64,1}:
 0.03963815566553452
 0.18809218402791444
 3.1080524224354096

In [21]:
AsymEig.vectors

3×3 Array{Float64,2}:
 -0.536078   0.688581  -0.48834
 -0.157035  -0.649733  -0.743766
  0.829434   0.32203   -0.456439

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [22]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
 1.0          -1.77636e-15  -2.66454e-15
 0.0           1.0           0.0
 1.77636e-15   7.10543e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [23]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [24]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [25]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.728887321912701

In [26]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [27]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [28]:
@time eigvals(Asym);

  0.356193 seconds (150.82 k allocations: 15.454 MiB, 3.35% gc time)


In [29]:
@time eigvals(Asym_noisy);

  1.083032 seconds (13 allocations: 7.920 MiB)


In [30]:
@time eigvals(Asym_explicit);

  0.247789 seconds (6.82 k allocations: 8.354 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [31]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.521242 seconds (453.04 k allocations: 206.250 MiB, 11.01% gc time)


6.476023928990831

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [32]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [33]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 9//10  2//5   1//5
 1//1   1//2   4//5
 9//10  3//10  1//10

In [34]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
  3//2
 23//10
 13//10

In [35]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [36]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1   0//1  0//1
 9//10  1//1  0//1
 9//10  1//3  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 1//1   1//2     4//5
 0//1  -3//20  -31//50
 0//1   0//1   -47//150

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [56]:
using LinearAlgebra

In [38]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [50]:
A_eigv = eigvals(A)

5-element Array{Float64,1}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [51]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [64]:
A_diag = Matrix(Diagonal(A_eigv))

5×5 Array{Float64,2}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [65]:
A_lowertri = Matrix(LowerTriangular(A))

5×5 Array{Int64,2}:
 140    0    0    0   0
  97  106    0    0   0
  74   89  152    0   0
 168  131  144   54   0
 131   36   71  142  36

In [67]:
# we use approximation because the float number have not exact decimals
@assert isapprox(A_diag,[-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468], atol=1e-3)
# #### 11.3
# Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]